## Veamos una forma distinta para abordar el problema de particiones:

Si $S = \{a_1, a_2, \dots, a_N\}$, asociamos a cada $a_i$ dos variables binarias:
$$x_i = 1 \iff a_i \in S_1 $$
$$y_i = 1 \iff a_i \in S_2$$

Tenemos que agregar al modelo la restriccion de que:

$$x_i + y_i = 1$$

Esta restriccion la podemos representar con la siguiente matriz:

 \begin{pmatrix}
    -I_N & I_N & \\
    I_N & -I_N &
  \end{pmatrix}

Donde $I_N$ es la matriz identidad de $N \times N$

Las variables las ponemos en el siguiente vector: 

$$v = [x_1, x_2, \dots, x_N, y_1, y_2, \dots, y_N]$$

Así, en la mitad izquierda de $v$ tendremos al conjunto $S_1$, y en la mitad derecha al conjunto $S_2$


In [ ]:
import os
import time
import numpy as np
import pandas as pd
import math
from utils import *
import dimod
import warnings
import matplotlib.pyplot as plt
from functions_dwave import *
warnings.filterwarnings('ignore')

In [ ]:
# Creamos una instancia del problema
set = np.array([3, 1, 1, 2, 2, 1])
set = np.array([7, 15, 3, 2, 5, 5, 10, 7])
set = np.array([11, 5, 1, 5]) # Este array solo puede ser particionado de una forma
set = np.array([1, 1, 1, 1, 1, 5, 10])
#set = np.array([1, 2, 3, 4, 10])
suma = sum(set)
size = len(set)
data = dict({'set': set})
df = pd.DataFrame(data)
print("-------------------------------------")
print("Partition the set: \n ")
print(df)
print("With total sum of: ", suma)

In [ ]:
id = np.identity(size)
q_restriccion = np.row_stack((np.column_stack((-id, id)), np.column_stack((id, -id))))

# Q_objetivo es la misma que en el modelo mas simple, solo agregamos ceros al final
# para que las dimensiones de las dos matrices coincidan.
vect = np.r_[set, np.zeros(size)]
q_objetivo = np.outer(vect, vect) - suma * np.diag(vect)


alpha = 1 # Elegimos una constante para el peso. Eventualmente hay que darle mas pienso
beta = 1
qubo = beta * q_objetivo + alpha * q_restriccion
print("qubo: \n", qubo)

In [ ]:
# Enviamos problema a Dwave:
print("simmulating....")
# aggregate = True, va a agrupar soluciones iguales. Retorna tambien las ocurrencias de cada solucion
sampleset = sendToDwave(qubo, 100, True)
print("Filtering:")
# Nos quedamos con la solucion que minimiza la energia:
energies = [element[1] for element in sampleset]
solution = sampleset[energies.index(min(energies))][0] # Le saco la energia, solo me quedo con la solucion
set_1 = solution[0:size]
set_2 = solution[size:2*size]
print(set_1)
print(set_2)


plt.bar([(str(el[1])) for el in sampleset], [(el[1]) for el in sampleset])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
index_of_set_1 = np.where(set_1 == 1)[0] # retorna los indices de los items elegidos.
index_of_set_2 = np.where(set_2 == 1)[0]
output_df_1 = df.iloc[index_of_set_1]
output_df_2 = df.iloc[index_of_set_2]
print("-------------------------------------")
print("Choosen items in Set 1 are: ")
print(output_df_1)
print("-------------------------------------")
print("-------------------------------------")
print("Choosen items in Set 2 are: ")
print(output_df_2)
print("-------------------------------------")


sum_set_1 = np.dot(set, solution[0:size])
sum_set_2 = suma - sum_set_1
print("Suma del primer conjunto = ", sum_set_1)
print("Suma del segundo conjunto = ", sum_set_2)

verificacion_disjunta = np.dot(set_1, set_2) # Si es cero, los conjuntos son efectivamente disjuntos
verificacion_particion = np.sum(set_1) + np.sum(set_2) - size

if verificacion_disjunta != 0:
    print("ERROR: los conjuntos no son disjuntos")

if verificacion_particion != 0:
    print("ERROR: los conjuntos no particionan S")